In [1]:
import pandas as pd
# Set option to display all columns
pd.set_option('display.max_columns', None)


# import data

In [2]:
data = pd.read_parquet("outputs/dedupe_data_out.parquet")
data

,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,parent_record_id,records_hierarchy_copy,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last
0,crossref::10.21467/preprints.48,AIJR Preprints,crossref,10.21467/preprints.48,https://doi.org/10.21467/preprints.48,https://preprints.aijr.org/index.php/ap/prepri...,"Bird’s Eye View on the Diagnosis, Treatment, &...","Panchalingala, Sai Bhargavi",None,None,None,None,,,,,false,None,None,None,None,parent,2020-05-03,2020,None,parent,None,None,None,None,None
1,crossref::10.21467/preprints.43,AIJR Preprints,crossref,10.21467/preprints.43,https://doi.org/10.21467/preprints.43,https://preprints.aijr.org/index.php/ap/prepri...,Doxycycline and Minocycline Drugs as a Treatme...,"Mostafa, Mohamed",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-25,2020,None,parent,None,mohamed_mostafa,doxycycline and minocycline drugs as a treatme...,mostafa|m,None
2,crossref::10.21467/preprints.39,AIJR Preprints,crossref,10.21467/preprints.39,https://doi.org/10.21467/preprints.39,https://preprints.aijr.org/index.php/ap/prepri...,A Genetic Perspective of 2019-nCoV in Relation...,"Dasgupta, Rimjhim",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-16,2020,None,parent,None,dasgupta_rimjhim,a genetic perspective of 2019 ncov in relation...,dasgupta|r,None
3,crossref::10.21467/preprints.38,AIJR Preprints,crossref,10.21467/preprints.38,https://doi.org/10.21467/preprints.38,https://preprints.aijr.org/index.php/ap/prepri...,Marine Algae as a Natural Source for Antiviral...,"Musale, Amar S; G., Raja Krishna Kumar; Sapre,...",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-15,2020,None,parent,None,None,None,None,None
4,crossref::10.21467/preprints.36,AIJR Preprints,crossref,10.21467/preprints.36,https://doi.org/10.21467/preprints.36,https://preprints.aijr.org/index.php/ap/prepri...,Possible Prevention of COVID 19 by Using Linol...,"Subhash, Venkata; G, Raja Krishna Kumar; Sapre...",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-15,2020,None,parent,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8410089,openalex::W999325625,viXra,openalex,None,None,https://vixra.org/pdf/1409.0090v1.pdf,Three Objections to Modern Physics,Lubomir Vlcek,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2014-09-01,2014,None,parent,None,lubomir_vlcek,three objections to modern physics,vlcek|l,None
8410090,openalex::W999460032,viXra,openalex,None,None,https://vixra.org/abs/1112.0094,Particle Mass Ratios,DT Froedge,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2011-12-01,2011,None,parent,None,dt_froedge,particle mass ratios,froedge|d,None
8410091,openalex::W99967155,viXra,openalex,None,None,https://vixra.org/pdf/1406.0019v1.pdf,Quantum FFF Theory Proposals for Some Unsolved...,Leo Vuyk,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2014-06-01,2014,None,parent,None,leo_vuyk,quantum fff theory proposals for some unsolved...,vuyk|l,None
8410092,openalex::W999790414,viXra,openalex,None,None,https://vixra.org/pdf/1306.0105v3.pdf,Investigation of the Formalism of Particle Dyn...,Chi-Yi Chen,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2013-06-01,2013,None,parent,None,chen_chi_yi,investigation of the formalism of particle dyn...,chen|c,None


# dedupe on title+authors (+ optional year)

In [3]:
"""
Reproducible 2-pass dedupe pipeline (Exact pass -> Fuzzy pass) with:
- Strong-but-cheap title normalization (cached)
- 3 author signatures: tokenbag | last_initial | last
- Stage A strict (title + authors_fp) exact
- Stage B relaxed (shared authors overlap) within exact-title groups (optional per stage)
- Optional fuzzy title fallback (token containment) BLOCKED by authors_fp (+ optional year)
- Prefilter modes:
    * title_dup  : keep rows where cleaned title repeats (fast exact stages)
    * author_dup : keep rows where authors_fp repeats (enables fuzzy stages when titles differ)
    * none       : keep all eligible (debug)

Includes:
- Metrics counters per stage
- Summary printing + early stop
- Deterministic labeling
- Designed for speed + low false positives (especially with last_initial)

USAGE:
1) Define STAGES_EXACT and STAGES_FUZZY
2) Run:
   df_out, metrics = run_dedupe_pipeline_two_passes(
       df,
       stages_exact=STAGES_EXACT,
       stages_fuzzy=STAGES_FUZZY,
       early_stop_if_new_labels_lt=500,
       print_summary=True,
       return_all_metrics=True,
       servers=None,
       across_servers=True,
       use_year=False,
       choose_parent="oldest",
       prefilter=True,
       date_candidates=('date_first_seen',),
       hierarchy_col="records_hierarchy",
       parent_id_col="parent_record_id",
       group_id_col="dup_group_id",
       add_authors_fingerprint_col=True,
       add_title_clean_col=True,
   )
"""

import pandas as pd
import numpy as np
import re
import time
import unicodedata
from typing import Iterable, Optional, Dict, Any, Tuple, List

# ============================================================
# 0) Regex + NA helpers
# ============================================================
_WS = re.compile(r"\s+")
_PUNCT_ALL = re.compile(r"[^\w\s]", re.UNICODE)  # remove everything except word chars + spaces
NA_LIKE = {"", "none", "null", "nan", "n/a", "[]", "{}", "na"}


# ============================================================
# 1) Utility: pick a date column + record_id numeric fallback
# ============================================================
def _pick_first_existing(df: pd.DataFrame, candidates: Iterable[str]) -> Optional[str]:
    for c in candidates:
        if c in df.columns:
            return c
    return None


def _record_id_key(s: pd.Series) -> pd.Series:
    """Fast numeric key from record_id (extract first digits)."""
    digits = s.astype("string").str.extract(r"(\d+)")[0]
    return pd.to_numeric(digits, errors="coerce")


# ============================================================
# 2) Title normalization (cheap, high ROI) + token containment
# ============================================================
def _strip_accents_text(x: str) -> str:
    return "".join(
        c for c in unicodedata.normalize("NFKD", x) if not unicodedata.combining(c)
    )


def _clean_title_series_v2(s: pd.Series) -> pd.Series:
    """
    Strong-but-cheap title normalization:
      - lowercase
      - strip accents
      - remove punctuation -> spaces
      - collapse whitespace
    """
    s = s.astype("string").fillna("").str.strip().str.lower()
    s = s.where(~s.isin(list(NA_LIKE)), "")
    s = s.apply(_strip_accents_text)
    s = s.str.replace(_PUNCT_ALL, " ", regex=True)
    s = s.str.replace(_WS, " ", regex=True).str.strip()
    return s


def _title_tokens_from_clean(title_clean: str) -> List[str]:
    """Tokenize already-clean title into tokens; drop very short tokens (len < 2)."""
    if not title_clean:
        return []
    return [t for t in title_clean.split(" ") if len(t) >= 2]


def _containment_score(a_tokens: List[str], b_tokens: List[str]) -> float:
    """
    Containment score:
        |A ∩ B| / min(|A|, |B|)
    Good for small title differences when tokens still mostly match.
    """
    if not a_tokens or not b_tokens:
        return 0.0
    A, B = set(a_tokens), set(b_tokens)
    denom = min(len(A), len(B))
    if denom <= 0:
        return 0.0
    return len(A & B) / denom


# ============================================================
# 3) Author canonicalization (3 modes)
# ============================================================
def _strip_accents(s: str) -> str:
    return "".join(
        c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c)
    )


def _normalize_one_author_tokenbag(author: str) -> str:
    """
    Token-bag per author:
    - remove punctuation
    - split tokens
    - sort tokens within author
    - join with "_"
    """
    if not author:
        return ""
    a = _strip_accents(str(author)).lower().strip()
    if not a or a in NA_LIKE:
        return ""
    a = _PUNCT_ALL.sub(" ", a)
    a = _WS.sub(" ", a).strip()
    if not a:
        return ""
    toks = [t for t in a.split(" ") if t]
    if not toks:
        return ""
    toks = sorted(toks)
    return "_".join(toks)


def _normalize_one_author_last_initial(author: str) -> str:
    """
    Middle-ground signature: "last|first_initial"
    Rules:
      - If comma: "Last, First ..." -> last = first token before comma;
                                     initial = first token after comma (first-name token only)
      - If no comma: "First ... Last" -> last = last token; initial = first token
      - If we can't find an initial, return "" (reduces false positives)
    """
    if not author:
        return ""
    a = _strip_accents(str(author)).lower().strip()
    if not a or a in NA_LIKE:
        return ""

    if "," in a:
        left, right = a.split(",", 1)
        left = _PUNCT_ALL.sub(" ", left)
        right = _PUNCT_ALL.sub(" ", right)
        left = _WS.sub(" ", left).strip()
        right = _WS.sub(" ", right).strip()
        if not left:
            return ""
        last_toks = [t for t in left.split(" ") if t]
        if not last_toks:
            return ""
        last = last_toks[0]  # keep your "first token if multi-token surname" philosophy

        first_toks = [t for t in right.split(" ") if t]
        if not first_toks:
            return ""  # avoid false positives
        ini = first_toks[0][:1]
        return f"{last}|{ini}" if ini else ""
    else:
        a = _PUNCT_ALL.sub(" ", a)
        a = _WS.sub(" ", a).strip()
        toks = [t for t in a.split(" ") if t]
        if len(toks) < 2:
            return ""
        ini = toks[0][:1]
        last = toks[-1]
        return f"{last}|{ini}" if (ini and last) else ""


def _normalize_one_author_last(author: str) -> str:
    """Last-name-only signature (high recall, more false positives)."""
    if not author:
        return ""
    a = _strip_accents(str(author)).lower().strip()
    if not a or a in NA_LIKE:
        return ""

    if "," in a:
        left = a.split(",", 1)[0].strip()
        left = _PUNCT_ALL.sub(" ", left)
        left = _WS.sub(" ", left).strip()
        if not left:
            return ""
        toks = [t for t in left.split(" ") if t]
        if not toks:
            return ""
        return toks[0]
    else:
        a = _PUNCT_ALL.sub(" ", a)
        a = _WS.sub(" ", a).strip()
        toks = [t for t in a.split(" ") if t]
        if not toks:
            return ""
        return toks[-1]


def build_authors_fingerprint_series(authors_flat: pd.Series, mode: str) -> pd.Series:
    """
    Build author fingerprint per row:
      - split authors on ';'
      - normalize each author (depends on mode)
      - drop empties
      - dedupe within row
      - sort
      - join with ';'
    """
    if mode not in {"tokenbag", "last_initial", "last"}:
        raise ValueError("mode must be tokenbag | last_initial | last")

    s = authors_flat.astype("string").fillna("").str.strip()
    s = s.where(~s.str.lower().isin(list(NA_LIKE)), "")

    if mode == "tokenbag":
        norm_fn = _normalize_one_author_tokenbag
    elif mode == "last_initial":
        norm_fn = _normalize_one_author_last_initial
    else:
        norm_fn = _normalize_one_author_last

    def row_to_fp(x: str) -> str:
        if not x:
            return ""
        authors = [a.strip() for a in str(x).split(";") if a.strip()]
        norm = [norm_fn(a) for a in authors]
        norm = [z for z in norm if z]
        norm = sorted(set(norm))
        return ";".join(norm)

    return s.apply(row_to_fp)


def _author_tokens_from_fp(fp: str) -> List[str]:
    if not fp:
        return []
    return [t for t in fp.split(";") if t]


def _overlap_count(a_tokens: List[str], b_tokens: List[str]) -> int:
    if not a_tokens or not b_tokens:
        return 0
    return len(set(a_tokens) & set(b_tokens))


# ============================================================
# 4) Single-stage dedupe:
#    - Prefilter (title_dup/author_dup/none)
#    - Stage A strict (title + authors_fp) exact match
#    - Optional fuzzy title fallback (within same authors_fp)
#    - Optional Stage B relaxed (shared authors overlap) within exact-title groups
# ============================================================
def dedupe_title_authors_stage(
    df: pd.DataFrame,
    *,
    # stage config
    stage_name: str = "stage",
    authors_fp_mode: str = "tokenbag",         # tokenbag | last_initial | last

    # fuzzy config (title containment), executed only if enabled
    title_fuzzy_fallback: bool = False,
    min_title_tokens: int = 6,
    min_title_containment: float = 0.70,
    fuzzy_compare_strategy: str = "parent_only",  # parent_only | all_pairs_small (parent_only is safest/fastest)

    # relaxed (shared authors overlap) config (exact title only)
    relaxed_shared_authors: bool = True,
    min_authors_required: int = 2,
    min_shared_authors: int = 2,

    # prefilter strategy (important!)
    prefilter_mode: str = "title_dup",         # title_dup | author_dup | none
    prefilter: bool = True,                    # if False, skip ">=2" group filter (slower)

    # global options
    servers=None,
    across_servers: bool = True,
    use_year: bool = False,
    choose_parent: str = "oldest",             # oldest | most_recent
    overwrite_mode: str = "parent_only",       # any | parent_only | unlabeled_only

    # columns
    server_col: str = "server_name",
    record_id_col: str = "record_id",
    title_col: str = "title",
    authors_col: str = "authors_flat",
    year_col: str = "publication_year_first_seen",
    date_candidates: Tuple[str, ...] = ("date_first_seen",),

    hierarchy_col: str = "records_hierarchy",
    parent_id_col: str = "parent_record_id",
    group_id_col: str = "dup_group_id",

    # caching/debug columns
    add_authors_fingerprint_col: bool = True,
    authors_fingerprint_col: str = "authors_fp",
    add_title_clean_col: bool = True,
    title_clean_col: str = "title_clean_v2",

    return_metrics: bool = False,
) -> pd.DataFrame | Tuple[pd.DataFrame, Dict[str, Any]]:
    """
    One dedupe stage. Designed to be composed into a multi-stage pipeline.
    """

    t0 = time.perf_counter()

    metrics: Dict[str, Any] = {
        "stage_name": stage_name,
        "n_rows_df": int(len(df)),
        "n_candidates_initial": 0,
        "prefilter_mode": prefilter_mode,
        "prefilter_rows": 0,
        "prefilter_groups": 0,
        "work_rows_after_keys": 0,

        "stageA_groups": 0,
        "stageA_children_labeled": 0,

        "fuzzy_enabled": bool(title_fuzzy_fallback),
        "fuzzy_groups": 0,
        "fuzzy_pairs_checked": 0,
        "fuzzy_children_labeled": 0,

        "stageB_enabled": bool(relaxed_shared_authors),
        "stageB_title_groups": 0,
        "stageB_clusters": 0,
        "stageB_children_labeled": 0,

        "time_s": 0.0,
    }

    # ------------------------------------------------------
    # Ensure output cols exist
    # ------------------------------------------------------
    for c in (hierarchy_col, parent_id_col, group_id_col):
        if c not in df.columns:
            df[c] = pd.NA

    if add_authors_fingerprint_col and authors_fingerprint_col not in df.columns:
        df[authors_fingerprint_col] = pd.NA
    if add_title_clean_col and title_clean_col not in df.columns:
        df[title_clean_col] = pd.NA

    # ------------------------------------------------------
    # Eligibility
    # ------------------------------------------------------
    h = df[hierarchy_col]
    if overwrite_mode == "any":
        eligible = pd.Series(True, index=df.index)
    elif overwrite_mode == "parent_only":
        eligible = h.astype("string").str.lower().str.strip().eq("parent")
    elif overwrite_mode == "unlabeled_only":
        eligible = h.isna()
    else:
        raise ValueError("overwrite_mode must be any | parent_only | unlabeled_only")

    # server filter
    if servers is None:
        server_mask = pd.Series(True, index=df.index)
    elif isinstance(servers, str):
        server_mask = df[server_col].eq(servers)
    else:
        server_mask = df[server_col].isin(list(servers))

    m = eligible & server_mask
    metrics["n_candidates_initial"] = int(m.sum())
    if not m.any():
        metrics["time_s"] = time.perf_counter() - t0
        return (df, metrics) if return_metrics else df

    # ------------------------------------------------------
    # Prefilter: decide which indices to consider in this stage
    # ------------------------------------------------------
    if prefilter_mode == "title_dup":
        # title-based prefilter (fast for exact title stages)
        t_clean = df.loc[m, title_clean_col] if (add_title_clean_col and title_clean_col in df.columns and df.loc[m, title_clean_col].notna().any()) else None
        if t_clean is None:
            t_clean = _clean_title_series_v2(df.loc[m, title_col])
        vc = t_clean.value_counts()
        keep_idx = t_clean[t_clean.isin(vc[vc >= 2].index)].index
        metrics["prefilter_groups"] = int((vc >= 2).sum())

    elif prefilter_mode == "author_dup":
        # author-fp based prefilter (crucial for fuzzy pass; titles may differ)
        # compute fp only for m rows
        a_fp = build_authors_fingerprint_series(df.loc[m, authors_col], mode=authors_fp_mode)
        vc = a_fp.value_counts()
        keep_idx = a_fp[a_fp.isin(vc[vc >= 2].index)].index
        metrics["prefilter_groups"] = int((vc >= 2).sum())

    elif prefilter_mode == "none":
        keep_idx = df.index[m]
        metrics["prefilter_groups"] = 0

    else:
        raise ValueError("prefilter_mode must be title_dup | author_dup | none")

    metrics["prefilter_rows"] = int(len(keep_idx))
    if len(keep_idx) == 0:
        metrics["time_s"] = time.perf_counter() - t0
        return (df, metrics) if return_metrics else df

    # ------------------------------------------------------
    # Work subset + compute/attach cached normalization keys
    # ------------------------------------------------------
    cols_needed = [server_col, record_id_col, title_col, authors_col]
    if use_year:
        cols_needed.append(year_col)
    date_col = _pick_first_existing(df, date_candidates)
    if date_col:
        cols_needed.append(date_col)

    work = df.loc[keep_idx, cols_needed].copy()

    # Title clean (cache to df if asked)
    if add_title_clean_col:
        # compute for missing only (cheap)
        t_missing = df.loc[work.index, title_clean_col].isna()
        if t_missing.any():
            df.loc[work.index[t_missing], title_clean_col] = _clean_title_series_v2(df.loc[work.index[t_missing], title_col]).values
        work["_t"] = df.loc[work.index, title_clean_col].astype("string").fillna("")
    else:
        work["_t"] = _clean_title_series_v2(work[title_col])

    # Authors fp (mode-specific; cache into df column if asked)
    work["_a_fp"] = build_authors_fingerprint_series(work[authors_col], mode=authors_fp_mode)
    if add_authors_fingerprint_col:
        df.loc[work.index, authors_fingerprint_col] = work["_a_fp"].values

    # Year (optional)
    if use_year:
        y = pd.to_numeric(work[year_col], errors="coerce")
        y = y.where((y >= 1000) & (y <= 3000)).round().astype("Int64")
        work["_y"] = y
    else:
        work["_y"] = pd.NA

    # require non-empty keys
    if use_year:
        work = work[(work["_t"] != "") & (work["_a_fp"] != "") & work["_y"].notna()].copy()
    else:
        work = work[(work["_t"] != "") & (work["_a_fp"] != "")].copy()

    metrics["work_rows_after_keys"] = int(len(work))
    if work.empty:
        metrics["time_s"] = time.perf_counter() - t0
        return (df, metrics) if return_metrics else df

    # ------------------------------------------------------
    # Stage A STRICT: exact match on (title_clean + authors_fp [+year] [+server scope])
    # ------------------------------------------------------
    if use_year:
        strict_base = work["_t"] + "||" + work["_a_fp"] + "||" + work["_y"].astype("string")
    else:
        strict_base = work["_t"] + "||" + work["_a_fp"]

    if across_servers:
        work["_grp_strict"] = strict_base
    else:
        work["_grp_strict"] = work[server_col].astype("string") + "||" + strict_base

    strict = work
    if prefilter:
        vcg = work["_grp_strict"].value_counts()
        dup_keys = vcg[vcg >= 2].index
        strict = work[work["_grp_strict"].isin(dup_keys)].copy()

    metrics["stageA_groups"] = int(strict["_grp_strict"].nunique()) if not strict.empty else 0

    # sort keys for parent choice
    if date_col and date_col in strict.columns:
        strict["_dt"] = pd.to_datetime(strict[date_col], errors="coerce")
    else:
        strict["_dt"] = pd.NaT
    strict["_rid"] = _record_id_key(strict[record_id_col])

    if not strict.empty:
        if choose_parent == "oldest":
            strict = strict.sort_values(
                by=["_grp_strict", "_dt", "_rid"],
                ascending=[True, True, True],
                na_position="last",
            )
        elif choose_parent == "most_recent":
            strict = strict.sort_values(
                by=["_grp_strict", "_dt", "_rid"],
                ascending=[True, False, False],
                na_position="last",
            )
        else:
            raise ValueError("choose_parent must be oldest | most_recent")

        parents = strict.groupby("_grp_strict", sort=False).head(1)
        parent_rid_map = parents.set_index("_grp_strict")[record_id_col]
        parent_srv_map = parents.set_index("_grp_strict")[server_col]

        strict["_parent_rid"] = strict["_grp_strict"].map(parent_rid_map)
        strict["_parent_srv"] = strict["_grp_strict"].map(parent_srv_map)

        is_parent = strict[record_id_col].eq(strict["_parent_rid"])
        parent_idx = strict.index[is_parent]
        child_idx = strict.index[~is_parent]

        metrics["stageA_children_labeled"] = int(len(child_idx))

        df.loc[parent_idx, hierarchy_col] = "parent"
        df.loc[parent_idx, parent_id_col] = pd.NA
        df.loc[child_idx, hierarchy_col] = (
            "parent - duplicate (" + strict.loc[child_idx, "_parent_srv"].astype("string") + ")"
        )
        df.loc[child_idx, parent_id_col] = strict.loc[child_idx, "_parent_rid"].values

        # deterministic group id
        df.loc[strict.index, group_id_col] = (
            pd.util.hash_pandas_object(strict["_grp_strict"], index=False)
            .astype("uint64")
            .astype(str)
            .values
        )

    # ------------------------------------------------------
    # Fuzzy title fallback (BLOCKED by authors_fp [+year], only remaining eligible)
    # Important: this can find near-duplicate titles because we do NOT rely on title_dup.
    # ------------------------------------------------------
    if title_fuzzy_fallback:
        # remaining eligible after Stage A
        h2 = df[hierarchy_col]
        if overwrite_mode == "parent_only":
            eligible2 = h2.astype("string").str.lower().str.strip().eq("parent")
        elif overwrite_mode == "unlabeled_only":
            eligible2 = h2.isna()
        else:
            eligible2 = pd.Series(True, index=df.index)

        remain_idx = work.index.intersection(df.index[eligible2])
        wF = work.loc[remain_idx].copy()

        if not wF.empty:
            # block by authors_fp (+year) because authors are "more trustworthy"
            if use_year:
                wF["_grp_auth"] = wF["_a_fp"] + "||" + wF["_y"].astype("string")
            else:
                wF["_grp_auth"] = wF["_a_fp"]

            # keep only blocks with >=2 rows
            vc_auth = wF["_grp_auth"].value_counts()
            keep_auth = vc_auth[vc_auth >= 2].index
            wF = wF[wF["_grp_auth"].isin(keep_auth)].copy()

            metrics["fuzzy_groups"] = int(wF["_grp_auth"].nunique()) if not wF.empty else 0

            if not wF.empty:
                # date/rid for parent selection
                if date_col and date_col in wF.columns:
                    wF["_dt"] = pd.to_datetime(wF[date_col], errors="coerce")
                else:
                    wF["_dt"] = pd.NaT
                wF["_rid"] = _record_id_key(wF[record_id_col])

                # tokens cache per row (within this stage)
                tokens_map = {idx: _title_tokens_from_clean(wF.loc[idx, "_t"]) for idx in wF.index}

                for grp, g in wF.groupby("_grp_auth", sort=False):
                    if len(g) < 2:
                        continue

                    # gate: ignore titles with too few tokens
                    idxs = [idx for idx in g.index if len(tokens_map.get(idx, [])) >= min_title_tokens]
                    if len(idxs) < 2:
                        continue

                    gg = g.loc[idxs].copy()
                    if choose_parent == "oldest":
                        gg = gg.sort_values(by=["_dt", "_rid"], ascending=[True, True], na_position="last")
                    else:
                        gg = gg.sort_values(by=["_dt", "_rid"], ascending=[False, False], na_position="last")

                    if fuzzy_compare_strategy == "parent_only":
                        parent_idx = gg.index[0]
                        parent_tokens = tokens_map[parent_idx]
                        parent_rid = gg.loc[parent_idx, record_id_col]
                        parent_srv = gg.loc[parent_idx, server_col]

                        # ensure parent labeled
                        df.loc[parent_idx, hierarchy_col] = "parent"
                        df.loc[parent_idx, parent_id_col] = pd.NA

                        for idx in gg.index[1:]:
                            metrics["fuzzy_pairs_checked"] += 1
                            sc = _containment_score(parent_tokens, tokens_map[idx])
                            if sc >= min_title_containment:
                                df.loc[idx, hierarchy_col] = f"parent - duplicate ({parent_srv})"
                                df.loc[idx, parent_id_col] = parent_rid
                                df.loc[idx, group_id_col] = f"fuzzy::{authors_fp_mode}::{grp}"
                                metrics["fuzzy_children_labeled"] += 1

                    elif fuzzy_compare_strategy == "all_pairs_small":
                        # safer than global all-pairs; still can be heavy if blocks are large.
                        # We'll cluster by greedy expansion (bounded within block).
                        idxs2 = gg.index.tolist()
                        used = set()
                        for i in idxs2:
                            if i in used:
                                continue
                            used.add(i)
                            cluster = [i]
                            for j in idxs2:
                                if j in used:
                                    continue
                                metrics["fuzzy_pairs_checked"] += 1
                                sc = _containment_score(tokens_map[i], tokens_map[j])
                                if sc >= min_title_containment:
                                    used.add(j)
                                    cluster.append(j)

                            if len(cluster) >= 2:
                                # choose parent (oldest/most recent) within cluster
                                cldf = gg.loc[cluster].copy()
                                if choose_parent == "oldest":
                                    cldf = cldf.sort_values(by=["_dt", "_rid"], ascending=[True, True], na_position="last")
                                else:
                                    cldf = cldf.sort_values(by=["_dt", "_rid"], ascending=[False, False], na_position="last")

                                p_idx = cldf.index[0]
                                p_rid = cldf.loc[p_idx, record_id_col]
                                p_srv = cldf.loc[p_idx, server_col]
                                df.loc[p_idx, hierarchy_col] = "parent"
                                df.loc[p_idx, parent_id_col] = pd.NA
                                for cidx in cldf.index[1:]:
                                    df.loc[cidx, hierarchy_col] = f"parent - duplicate ({p_srv})"
                                    df.loc[cidx, parent_id_col] = p_rid
                                    df.loc[cidx, group_id_col] = f"fuzzy::{authors_fp_mode}::{grp}"
                                    metrics["fuzzy_children_labeled"] += 1
                    else:
                        raise ValueError("fuzzy_compare_strategy must be parent_only | all_pairs_small")

    # ------------------------------------------------------
    # Stage B RELAXED (shared authors overlap) within exact title groups
    # ------------------------------------------------------
    if relaxed_shared_authors:
        h3 = df[hierarchy_col]
        if overwrite_mode == "parent_only":
            eligible3 = h3.astype("string").str.lower().str.strip().eq("parent")
        elif overwrite_mode == "unlabeled_only":
            eligible3 = h3.isna()
        else:
            eligible3 = pd.Series(True, index=df.index)

        remain_idx = work.index.intersection(df.index[eligible3])
        w2 = work.loc[remain_idx].copy()
        if not w2.empty:
            if use_year:
                relaxed_base = w2["_t"] + "||" + w2["_y"].astype("string")
            else:
                relaxed_base = w2["_t"]

            if across_servers:
                w2["_grp_title"] = relaxed_base
            else:
                w2["_grp_title"] = w2[server_col].astype("string") + "||" + relaxed_base

            # keep only repeated titles
            vc2 = w2["_grp_title"].value_counts()
            keep_groups = vc2[vc2 >= 2].index
            w2 = w2[w2["_grp_title"].isin(keep_groups)].copy()

            metrics["stageB_title_groups"] = int(w2["_grp_title"].nunique()) if not w2.empty else 0

            if not w2.empty:
                w2["_a_tokens"] = w2["_a_fp"].apply(_author_tokens_from_fp)
                w2["_a_n"] = w2["_a_tokens"].apply(len)

                if date_col and date_col in w2.columns:
                    w2["_dt"] = pd.to_datetime(w2[date_col], errors="coerce")
                else:
                    w2["_dt"] = pd.NaT
                w2["_rid"] = _record_id_key(w2[record_id_col])

                group_counter = 0
                children_total = 0

                for grp, g in w2.groupby("_grp_title", sort=False):
                    if len(g) < 2:
                        continue

                    g = g[g["_a_n"] >= min_authors_required].copy()
                    if len(g) < 2:
                        continue

                    idxs = g.index.tolist()
                    used = set()
                    clusters = []

                    # simple greedy clustering based on author overlap
                    for i in idxs:
                        if i in used:
                            continue
                        used.add(i)
                        cl = [i]
                        for j in idxs:
                            if j in used:
                                continue
                            if _overlap_count(g.loc[i, "_a_tokens"], g.loc[j, "_a_tokens"]) >= min_shared_authors:
                                used.add(j)
                                cl.append(j)
                        if len(cl) >= 2:
                            clusters.append(cl)

                    for cl in clusters:
                        group_counter += 1
                        cldf = g.loc[cl].copy()
                        if choose_parent == "oldest":
                            cldf = cldf.sort_values(by=["_dt", "_rid"], ascending=[True, True], na_position="last")
                        else:
                            cldf = cldf.sort_values(by=["_dt", "_rid"], ascending=[False, False], na_position="last")

                        parent_idx = cldf.index[0]
                        parent_rid = cldf.loc[parent_idx, record_id_col]
                        parent_srv = cldf.loc[parent_idx, server_col]

                        df.loc[parent_idx, hierarchy_col] = "parent"
                        df.loc[parent_idx, parent_id_col] = pd.NA
                        df.loc[parent_idx, group_id_col] = f"relaxed::{stage_name}::{group_counter}"

                        child_idxs = [x for x in cldf.index if x != parent_idx]
                        children_total += len(child_idxs)

                        df.loc[child_idxs, hierarchy_col] = f"parent - duplicate ({parent_srv})"
                        df.loc[child_idxs, parent_id_col] = parent_rid
                        df.loc[child_idxs, group_id_col] = f"relaxed::{stage_name}::{group_counter}"

                metrics["stageB_clusters"] = int(group_counter)
                metrics["stageB_children_labeled"] = int(children_total)

    metrics["time_s"] = time.perf_counter() - t0
    return (df, metrics) if return_metrics else df


# ============================================================
# 5) Stage runner with summary + early stop
# ============================================================
def _count_children_labels(series: pd.Series) -> int:
    s = series.astype("string").fillna("")
    return int(s.str.startswith("parent - duplicate").sum())


def run_dedupe_stages(
    df: pd.DataFrame,
    *,
    stages: List[Dict[str, Any]],
    early_stop_if_new_labels_lt: int = 100,
    print_summary: bool = True,
    return_all_metrics: bool = True,
    # common kwargs passed to every stage
    **common_kwargs,
) -> Tuple[pd.DataFrame, List[Dict[str, Any]]] | pd.DataFrame:
    """
    Runs a list of stages sequentially with:
      - delta duplicates added per stage
      - early stop
    """
    df_out = df
    metrics_all: List[Dict[str, Any]] = []

    prev_children = _count_children_labels(df_out[common_kwargs.get("hierarchy_col", "records_hierarchy")])

    for stage in stages:
        name = stage.get("name", stage.get("stage_name", "stage"))
        t0 = time.perf_counter()

        df_out, m = dedupe_title_authors_stage(
            df_out,
            return_metrics=True,
            stage_name=name,
            **common_kwargs,
            **{k: v for k, v in stage.items() if k not in {"name", "stage_name"}},
        )

        now_children = _count_children_labels(df_out[common_kwargs.get("hierarchy_col", "records_hierarchy")])
        delta = now_children - prev_children
        prev_children = now_children

        m["stage_runtime_s"] = time.perf_counter() - t0
        m["new_children_added"] = int(delta)
        metrics_all.append(m)

        if print_summary:
            print(
                f"[{name}] new_children={delta} | "
                f"cand={m['n_candidates_initial']} | "
                f"prefilter_rows={m['prefilter_rows']} | "
                f"A_children={m['stageA_children_labeled']} | "
                f"fuzzy_children={m['fuzzy_children_labeled']} | "
                f"B_children={m['stageB_children_labeled']} | "
                f"time={m['stage_runtime_s']:.2f}s"
            )

        if delta < early_stop_if_new_labels_lt:
            if print_summary:
                print(f"Early stop after {name}: delta {delta} < {early_stop_if_new_labels_lt}")
            break

    return (df_out, metrics_all) if return_all_metrics else df_out


# ============================================================
# 6) Two-pass pipeline: Exact pass -> Fuzzy pass on remaining parents
# ============================================================
def run_dedupe_pipeline_two_passes(
    df: pd.DataFrame,
    *,
    stages_exact: List[Dict[str, Any]],
    stages_fuzzy: List[Dict[str, Any]],
    early_stop_if_new_labels_lt: int = 100,
    print_summary: bool = True,
    return_all_metrics: bool = True,
    **common_kwargs,
) -> Tuple[pd.DataFrame, List[Dict[str, Any]]] | pd.DataFrame:
    """
    Pass A: run stages_exact (typically no fuzzy, prefilter_mode=title_dup).
    Pass B: run stages_fuzzy (fuzzy enabled, prefilter_mode=author_dup), on remaining parents only.

    IMPORTANT:
      - For Pass A, it is normal to use overwrite_mode="any" for stage1, then "parent_only" for stage2-3.
      - For Pass B, use overwrite_mode="parent_only" so we only touch unresolved parents.
    """
    all_metrics: List[Dict[str, Any]] = []
    df_out = df

    if print_summary:
        print("\n=== PASS A: EXACT ===")

    df_out, mA = run_dedupe_stages(
        df_out,
        stages=stages_exact,
        early_stop_if_new_labels_lt=early_stop_if_new_labels_lt,
        print_summary=print_summary,
        return_all_metrics=True,
        **common_kwargs,
    )
    all_metrics.extend(mA)

    if print_summary:
        print("\n=== PASS B: FUZZY (remaining parents) ===")

    df_out, mB = run_dedupe_stages(
        df_out,
        stages=stages_fuzzy,
        early_stop_if_new_labels_lt=early_stop_if_new_labels_lt,
        print_summary=print_summary,
        return_all_metrics=True,
        **common_kwargs,
    )
    all_metrics.extend(mB)

    return (df_out, all_metrics) if return_all_metrics else df_out


# ============================================================
# 7) Default stage configs (recommended)
# ============================================================

# PASS A (EXACT) — fast + high precision
STAGES_EXACT = [
    dict(
        name="A1_tokenbag_exact",
        authors_fp_mode="tokenbag",
        prefilter_mode="title_dup",
        title_fuzzy_fallback=False,
        relaxed_shared_authors=True,
        min_authors_required=1,
        min_shared_authors=1,
        overwrite_mode="parent_only",
        authors_fingerprint_col="authors_fp_tokenbag",
    ),
    dict(
        name="A2_last_initial_exact",
        authors_fp_mode="last_initial",
        prefilter_mode="title_dup",
        title_fuzzy_fallback=False,
        relaxed_shared_authors=True,
        min_authors_required=1,
        min_shared_authors=1,
        overwrite_mode="parent_only",
        authors_fingerprint_col="authors_fp_last_initial",
    ),
    dict(
        name="A3_last_exact_strict",
        authors_fp_mode="last",
        prefilter_mode="title_dup",
        title_fuzzy_fallback=False,
        relaxed_shared_authors=False,  # last-only already high recall; keep strict
        overwrite_mode="parent_only",
        authors_fingerprint_col="authors_fp_last",
    ),
]

# PASS B (FUZZY) — only remaining parents; block by authors_fp repetition
# Note: relaxed_shared_authors is usually OFF here to keep false positives down.
STAGES_FUZZY = [
    dict(
        name="B1_tokenbag_fuzzy",
        authors_fp_mode="tokenbag",
        prefilter_mode="author_dup",
        title_fuzzy_fallback=True,
        min_title_tokens=6,
        min_title_containment=0.50,  # start conservative; lower = more recall, more risk
        fuzzy_compare_strategy="parent_only",
        relaxed_shared_authors=False,
        # min_authors_required=1,
        # min_shared_authors=1,
        overwrite_mode="parent_only",
        authors_fingerprint_col="authors_fp_tokenbag",
    ),
    dict(
        name="B2_last_initial_fuzzy",
        authors_fp_mode="last_initial",
        prefilter_mode="author_dup",
        title_fuzzy_fallback=True,
        min_title_tokens=6,
        min_title_containment=0.70,
        fuzzy_compare_strategy="parent_only",
        relaxed_shared_authors=False,
        # min_authors_required=1,
        # min_shared_authors=1,
        overwrite_mode="parent_only",
        authors_fingerprint_col="authors_fp_last_initial",
    ),
]

# ============================================================
# 8) Example usage
# ============================================================
# df_out, metrics = run_dedupe_pipeline_two_passes(
#     df,
#     stages_exact=STAGES_EXACT,
#     stages_fuzzy=STAGES_FUZZY,
#     early_stop_if_new_labels_lt=500,
#     print_summary=True,
#     return_all_metrics=True,
#     servers=None,
#     across_servers=True,
#     use_year=False,
#     choose_parent="oldest",
#     prefilter=True,
#     date_candidates=('date_first_seen',),
#     hierarchy_col="records_hierarchy",
#     parent_id_col="parent_record_id",
#     group_id_col="dup_group_id",
#     add_authors_fingerprint_col=True,
#     add_title_clean_col=True,
#     title_clean_col="title_clean_v2",
# )
#
# print(metrics[-1])
# print(df_out["records_hierarchy"].value_counts(dropna=False).head(60))


In [4]:
data['records_hierarchy_copy2'] = data['records_hierarchy']

In [5]:
data["records_hierarchy_copy2"].value_counts(dropna=False).head(60)

records_hierarchy_copy2
parent                                                        7442983
parent - duplicate (SSRN)                                      176436
review                                                         152918
part_of                                                        149382
parent - duplicate (Zenodo)                                     94301
parent - duplicate (arXiv)                                      61928
parent - duplicate (Research Square)                            43399
parent - duplicate (RePEc: Research Papers in Economics)        32214
parent - duplicate (HAL)                                        30485
parent - duplicate (bioRxiv)                                    26023
version                                                         20174
parent - duplicate (Open Science Framework)                     18028
parent - duplicate (ResearchGate)                               15494
parent - duplicate (AgEcon Search)                              14

In [6]:
data["records_hierarchy"].value_counts(dropna=False).head(60)

records_hierarchy
parent                                                        7442983
parent - duplicate (SSRN)                                      176436
review                                                         152918
part_of                                                        149382
parent - duplicate (Zenodo)                                     94301
parent - duplicate (arXiv)                                      61928
parent - duplicate (Research Square)                            43399
parent - duplicate (RePEc: Research Papers in Economics)        32214
parent - duplicate (HAL)                                        30485
parent - duplicate (bioRxiv)                                    26023
version                                                         20174
parent - duplicate (Open Science Framework)                     18028
parent - duplicate (ResearchGate)                               15494
parent - duplicate (AgEcon Search)                              14431
pa

In [ ]:
data_out, metrics = run_dedupe_pipeline_two_passes(
    data,
    stages_exact=STAGES_EXACT,
    stages_fuzzy=STAGES_FUZZY,
    early_stop_if_new_labels_lt=1,
    print_summary=True,
    return_all_metrics=True,
    servers=None,
    across_servers=True,
    use_year=False,
    choose_parent="oldest",
    prefilter=True,
    date_candidates=('date_first_seen',),
    hierarchy_col="records_hierarchy",
    parent_id_col="parent_record_id",
    group_id_col="dup_group_id",
    add_authors_fingerprint_col=True,
    add_title_clean_col=True,
    title_clean_col="title_clean_v2",
)

print(metrics[-1])
print(data_out["records_hierarchy"].value_counts(dropna=False).head(60))



=== PASS A: EXACT ===
[A1_tokenbag_exact] new_children=3 | cand=7442983 | prefilter_rows=58420 | A_children=0 | fuzzy_children=0 | B_children=3 | time=16.27s
[A2_last_initial_exact] new_children=3 | cand=7442980 | prefilter_rows=58417 | A_children=0 | fuzzy_children=0 | B_children=3 | time=16.61s
[A3_last_exact_strict] new_children=0 | cand=7442977 | prefilter_rows=58411 | A_children=0 | fuzzy_children=0 | B_children=0 | time=5.52s
Early stop after A3_last_exact_strict: delta 0 < 1

=== PASS B: FUZZY (remaining parents) ===


In [ ]:
from pathlib import Path
import json
import pandas as pd

OUT_DIR = Path("outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Save metrics
with open(OUT_DIR / "dedupe_metrics_2.json", "w") as f:
    json.dump(metrics, f, indent=2)

pd.DataFrame(metrics).to_csv(OUT_DIR / "dedupe_metrics_2.csv", index=False)

# Save dataset
data_out.to_parquet(OUT_DIR / "dedupe_data_out_2.parquet", index=False)

print("✅ All files saved to outputs/")


In [ ]:
from pathlib import Path

OUT_DIR = Path("outputs")
OUT_DIR.mkdir(exist_ok=True)

new_cols = [
    "records_hierarchy_copy2",
    "records_hierarchy_copy",
    "records_hierarchy",
    "parent_record_id",
    "dup_group_id",
    "authors_fp_tokenbag",
    "authors_fp_last_initial",
    "authors_fp_last",
    "title_clean_v2",
]
# original_cols = set(df.columns)

# detect new columns
# new_cols = [c for c in data_out.columns if c not in original_cols]

cols_to_save = ["record_id"] + new_cols

data_out[cols_to_save].to_parquet(
    OUT_DIR / "dedupe_data_out_new_cols_2.parquet",
    index=False
)
data_out[cols_to_save].to_csv(
    OUT_DIR / "dedupe_data_out_new_cols_2.csv",
    index=False
)
print("✅ Saved:", cols_to_save)



In [ ]:
metrics

In [ ]:
data_out

In [ ]:
print(data_out["records_hierarchy_copy"].value_counts(dropna=False).head(60))

In [ ]:
print(data_out["parent_record_id"].value_counts(dropna=False).head(60))

In [ ]:
pattern = "fuzzy::las"


mask = data_out['dup_group_id'].str.contains(pattern, regex=False, na=False)
result = data_out[mask]
print(len(result))
result

In [ ]:
result['relations_json'][281751]

In [ ]:
data_out[data_out["record_id"]=='crossref::10.26434/chemrxiv-2021-cj17s']

In [ ]:
data_out[data_out["parent_record_id"]=='crossref::10.26434/chemrxiv-2022-fd190']

In [ ]:
pattern = "chemrxiv.11846943"


mask = data_out['doi'].str.contains(pattern, regex=False, na=False)
result = data_out[mask]
print(len(result))
result

In [ ]:
result['records_hierarchy'].value_counts()

In [ ]:
result['title'].value_counts()

In [ ]:
result['authors_flat'].value_counts()

In [ ]:
pattern = "10.26434/chemrxiv-2022-fd190"


mask = data_out['doi'].str.contains(pattern, regex=False, na=False)
result = data_out[mask]
print(len(result))
result

In [ ]:
pattern = "10.26434/chemrxiv-2022-fd190"


mask = data['doi'].str.contains(pattern, regex=False, na=False)
result = data[mask]
print(len(result))
result

In [ ]:
result['title'].value_counts()

In [ ]:
result['authors_flat'].value_counts()

In [ ]:
sample_titles = data_out.sample(5)[['title', 'authors_flat']].title
data_out[data_out.title.isin(sample_titles)][['title','authors_flat','records_hierarchy','date_first_seen']]

In [ ]:
pattern = "10.26434/chemrxiv.13102877"


mask = data_out['doi'].str.contains(pattern, regex=False, na=False)
result = data_out[mask]
print(len(result))
result